In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference_df, analysis_df, _ = nml.load_synthetic_car_loan_dataset()
display(reference_df.head())

,id,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


In [ ]:
print(reference_df.head().to_markdown(tablefmt="grid"))

+----+------+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   id |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+======+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |    0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+------+-------------+----------------+------------------------+---------------+---------------------------+---

In [ ]:
# Define feature columns
feature_column_names = [
    'car_value',
    'salary_range',
    'debt_to_income_ratio',
    'loan_length',
    'repaid_loan_on_prev_car',
    'size_of_downpayment',
    'driver_tenure'
]

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
calc.fit(reference_df)
results = calc.calculate(analysis_df)

In [ ]:
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                    reconstruction_error                 \
                 end_date    period                value sampling_error   
0 2018-11-30 00:27:16.848  analysis             1.141517       0.007025   
1 2018-12-30 07:03:16.848  analysis             1.130636       0.007098   
2 2019-01-29 13:39:16.848  analysis             1.138914       0.006977   
3 2019-02-28 20:15:16.848  analysis             1.145040       0.007077   
4 2019-03-31 02:51:16.848  analysis             1.137563       0.006994   
5 2019-04-30 09:27:16.848  analysis             1.249205       0.007036   
6 2019-05-30 16:03:16.848  analysis             1.243101       0.006985   
7 2019-06-29 22:39:16.848  analysis             1.258147       0.006960   
8 2019-07-30 05:15:16.848  analysis             1.228179       0.006967   
9 2019-08-29 11:51:16.848  analysis             1.259884       0.006966   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.162592                  1.120442        1.153019   
1                  1.151932                  1.109341        1.153019   
2                  1.159846                  1.117982        1.153019   
3                  1.166271                  1.123809        1.153019   
4                  1.158544                  1.116582        1.153019   
5                  1.270314                  1.228097        1.153019   
6                  1.264057                  1.222146        1.153019   
7                  1.279028                  1.237266        1.153019   
8                  1.249080                  1.207277        1.153019   
9                  1.280781                  1.238986        1.153019   

                          
  lower_threshold  alert  
0        1.115281  False  
1        1.115281  False  
2        1.115281  False  
3        1.115281  False  
4        1.115281  False  
5        1.115281   True  
6        1.115281   True  
7        1.115281   True  
8        1.115281   True  
9        1.115281   True

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+--------------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | reconstruction_error   |                    |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | value                  | | sampling_error   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====

In [ ]:
display(results.filter(period='reference').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-01-01 00:00:00   
1    [5000:9999]           1        5000      9999 2018-01-31 06:36:00   
2  [10000:14999]           2       10000     14999 2018-03-02 13:12:00   
3  [15000:19999]           3       15000     19999 2018-04-01 19:48:00   
4  [20000:24999]           4       20000     24999 2018-05-02 02:24:00   
5  [25000:29999]           5       25000     29999 2018-06-01 09:00:00   
6  [30000:34999]           6       30000     34999 2018-07-01 15:36:00   
7  [35000:39999]           7       35000     39999 2018-07-31 22:12:00   
8  [40000:44999]           8       40000     44999 2018-08-31 04:48:00   
9  [45000:49999]           9       45000     49999 2018-09-30 11:24:00   

                                     reconstruction_error                 \
                 end_date     period                value sampling_error   
0 2018-01-31 06:27:16.848  reference             1.136408       0.007005   
1 2018-03-02 13:03:16.848  reference             1.134478       0.006933   
2 2018-04-01 19:39:16.848  reference             1.136076       0.007078   
3 2018-05-02 02:15:16.848  reference             1.140114       0.007019   
4 2018-06-01 08:51:16.848  reference             1.126078       0.006874   
5 2018-07-01 15:27:16.848  reference             1.142018       0.007046   
6 2018-07-31 22:03:16.848  reference             1.125146       0.006981   
7 2018-08-31 04:39:16.848  reference             1.143206       0.007066   
8 2018-09-30 11:15:16.848  reference             1.130954       0.006957   
9 2018-10-30 17:51:16.848  reference             1.127026       0.007001   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.157422                  1.115394        1.153019   
1                  1.155278                  1.113678        1.153019   
2                  1.157309                  1.114844        1.153019   
3                  1.161173                  1.119056        1.153019   
4                  1.146701                  1.105456        1.153019   
5                  1.163155                  1.120881        1.153019   
6                  1.146089                  1.104204        1.153019   
7                  1.164406                  1.122007        1.153019   
8                  1.151825                  1.110082        1.153019   
9                  1.148028                  1.106024        1.153019   

                          
  lower_threshold  alert  
0        1.115281  False  
1        1.115281  False  
2        1.115281  False  
3        1.115281  False  
4        1.115281  False  
5        1.115281  False  
6        1.115281  False  
7        1.115281  False  
8        1.115281  False  
9        1.115281  False

In [ ]:
print_multi_index_markdown(results.filter(period='reference').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+--------------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | reconstruction_error   |                    |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | value                  | | sampling_error   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====

In [ ]:
figure = results.plot()
figure.show()

In [ ]:
figure.write_image('../_static/tutorials/detecting_data_drift/multivariate_drift_detection/pca-reconstruction-error.svg')

In [ ]:
non_feature_columns = ['timestamp', 'y_pred_proba', 'y_pred', 'repaid']

feature_column_names = [
    col for col in reference_df.columns
    if col not in non_feature_columns
]

from sklearn.impute import SimpleImputer

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000,
    imputer_categorical=SimpleImputer(strategy='constant', fill_value='missing'),
    imputer_continuous=SimpleImputer(strategy='median')
)
calc.fit(reference_df)
results = calc.calculate(analysis_df)
